In [155]:
import pinyin
import re
from collections import Counter

In [156]:
def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)

In [157]:
def chinese_to_pinyin(character):
    ret = []
    character_clean = [''.join(token(str(a))) for a in character]
    for word in character_clean:
        ret += [pinyin.get(word,format='strip', delimiter=' ')]
    return ret

In [158]:
def get_token():
    ret = []
    with open( './article_9k.txt', 'r' ) as f:
        for line in f.readlines():
            ret += chinese_to_pinyin(line)
            break
    return ret

In [159]:
def pinyin_counter(token):
    return Counter(token)

In [160]:
TOKEN = get_token()
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]
TOKEN_2_GRAM[:10]


['ciwai',
 'waizi',
 'ziben',
 'benzhou',
 'zhou6',
 '6yue',
 'yue1',
 '12',
 '2ri',
 'riqi']

In [223]:
words_count = pinyin_counter(TOKEN)
words_count.most_common(10)

[('I', 6),
 ('yi', 5),
 ('shi', 5),
 ('ji', 4),
 ('fa', 4),
 ('bu', 4),
 ('you', 4),
 ('xing', 3),
 ('yu', 3),
 ('xin', 3)]

In [224]:
words_count_2 = Counter(TOKEN_2_GRAM)

In [225]:
def prob_1(word):
    return (words_count[word] + 1) / len(TOKEN)

In [226]:
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: 
        return words_count_2[word1+word2] / len(TOKEN_2_GRAM)
    else:
        return 1 / len(TOKEN_2_GRAM)

In [233]:
def split_words(spell):
    left_word = ''
    right_word = ''
    sentence = ''
    list_sentence = []
    prob = 1
    for i,w in enumerate(spell):
        left_word = left_word + w
        right_word = spell[i+1:]
        
        prob = prob_1(left_word)
        print(left_word)
        print(right_word)
        (sub_sen, sub_prob) = split_words(right_word)
        sentence = left_word + ' ' + sub_sen
        list_sentence += [(sentence, prob)]
        
    return max(list_sentence, key=lambda x: x[1])

In [234]:
(sentence, prob) = split_words('woaibeijing')
print(sentence)
print(prob)

w
oaibeijing
o
aibeijing
a
ibeijing
i
beijing
b
eijing
e
ijing
i
jing
j
ing
i
ng
n
g
g



ValueError: max() arg is an empty sequence